# Classification Tree

In [110]:
import pandas as pd
import numpy as np
import glob as glob
import json
import sklearn
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from random import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, explained_variance_score


import keras
from keras.layers import Dense, Dropout, RepeatVector, BatchNormalization, Convolution1D, Flatten, Lambda, Permute, MaxPooling1D, AlphaDropout
from keras.models import Sequential
from keras.utils import to_categorical
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [2]:
root_path = 'features_by_valence/'
label_set = ['low', 'high']
low_files = glob.glob(root_path + label_set[0] + '/*')
high_files = glob.glob(root_path + label_set[1] + '/*')

features = {'low':[], 'high':[]}

all_features = []

for filename in low_files:
    word_histogram = json.loads(open(filename).read())
    features['low'].append(word_histogram)
    all_features.append({'dictionary': word_histogram, 'label': 'low'})
for filename in high_files:
    word_histogram = json.loads(open(filename).read())
    features['high'].append(word_histogram)
    all_features.append({'dictionary': word_histogram, 'label': 'high'})

In [3]:
n_total = int(len(all_features))
n_train = int(n_total * 0.8)
n_test = int(n_total * 0.2)

print('total: ' + str(n_total))
print('train: ' + str(n_train))
print('test: ' + str(n_test))

total: 252
train: 201
test: 50


In [23]:
# one hot encoding

def word_list(percent_occurance):
    key_set = {}

    for feature_d in all_features:
        keys = feature_d['dictionary'].keys()
        for key in keys:
            key_set[key] = key_set.get(key, 0) + 1
    key_set
    # Return words that occur in the top percent_occurance% of documents
    key_set = dict([[key, key_set[key]] for key in key_set if (key_set[key]/float(n_total) > float(percent_occurance))])

    words_sorted_alphabetically = list(key_set.keys())
    words_sorted_alphabetically.sort()
    return words_sorted_alphabetically

In [5]:
def get_index(reference, key):
    try:
        return reference.index(key)
    except:
        return len(reference)

In [6]:
ohe = {}
i_2_label = {}

def one_hot_encode(reference, dictionary):
    ohe = np.zeros(len(reference) + 1)
    for key in dictionary.keys():
        pos = get_index(reference, key)
        ohe[pos] = dictionary[key]
    norm_lookup = float(np.linalg.norm(ohe))
    normalized_ohe = [float(item)/float(norm_lookup) for item in ohe]
    return ohe

In [7]:
def encode(y):
    if y == 'high':
        return 1
    else:
        return 0
    
def decode(y):
    if y == 1:
        return 'high'
    else:
        return 'low'

In [8]:
def validate(af):
    classifications = {'low':0, 'high':0}
    correct_preds = 0
    all_preds = 0
    for d_item in af:
        feature = d_item['ohe_feature']
        label = d_item['ohe_label']
        classification = clf.predict([feature])[0]
        classification = decode(classification)
        classifications[classification] = classifications[classification] + 1
        if classification == label:
            correct_preds = correct_preds + 1
        all_preds = all_preds + 1
    print(classifications)
    return float(correct_preds)/float(all_preds)

In [9]:
def preds_and_true(clf, af, ohe_words):
    correct_preds = 0
    all_preds = 0
    preds = []
    true = []
    for d_item in af:
        feature = d_item['ohe_feature']
        label = d_item['ohe_label']
        classification = clf.predict([feature])[0]
        preds.append(classification)
        true.append( label )
    return [preds, true]

In [10]:
def encode_features_and_words(all_features, word_threshold):
    ohe_words = word_list(word_threshold)
    
    for i, f in enumerate(all_features):
        ohe_feature = one_hot_encode(ohe_words, f['dictionary'])
        ohe_label = encode(f['label'])
        all_features[i]['ohe_feature'] = ohe_feature
        all_features[i]['ohe_label'] = ohe_label
    return [all_features, ohe_words]


In [128]:
def model(all_features, ohe_words):
    rounds = []
    for i in range(20):
        shuffle(all_features)
        
        train_features = all_features[0:n_train]
        test_features = all_features[n_train:n_total]
    
        X = [f['ohe_feature'] for f in train_features]
        y = [f['ohe_label'] for f in train_features]
        
        clf = GradientBoostingClassifier(random_state=0, learning_rate=0.005)
        clf.fit(X, y)
        
        preds, true = preds_and_true(clf, train_features, ohe_words)
        train_precision = precision_score(true, preds)
        train_acc = accuracy_score(true, preds)
        tr_ev = explained_variance_score(true,preds)
        
        preds, true = preds_and_true(clf, test_features, ohe_words)
        test_precision = precision_score(true, preds)
        test_acc = accuracy_score(true, preds)
        te_ev = explained_variance_score(true, preds)
        
        rounds.append({
            'train_precision': train_precision, 
            'train_acc': train_acc, 
            'tr_ev': tr_ev,
            'test_precision': test_precision,
            'test_acc': test_acc, 
            'te_ev': te_ev
            
        })
    totals = {}
    output = {}
    for ro in rounds: 
        for k in ro:
            ro_total_key = 'total-' + k
            totals[ro_total_key] = totals.get(ro_total_key, 0) + ro[k]
    for k in totals:
        ou_av_key = 'av-' + k
        output[ou_av_key] = (totals[k]/float(len(rounds)))
    return output
        

In [129]:
n = 0.25
all_features, ohe_words = encode_features_and_words(all_features, n)
model(all_features, ohe_words)

{'av-total-te_ev': -0.60559675360554177,
 'av-total-test_acc': 0.53921568627450978,
 'av-total-test_precision': 0.46470682095682098,
 'av-total-tr_ev': 0.68441162543144385,
 'av-total-train_acc': 0.91567164179104488,
 'av-total-train_precision': 0.99180114076448278}

In [45]:
clf = GradientBoostingClassifier(random_state=0, learning_rate=0.005)
clf.fit(X, y)

preds, true = preds_and_true(clf, train_features, ohe_words)
train_precision = precision_score(true, preds)
train_acc = accuracy_score(true, preds)
tr_ev = explained_variance_score(true,preds)

preds, true = preds_and_true(clf, test_features, ohe_words)
test_precision = precision_score(true, preds)
test_acc = accuracy_score(true, preds)
te_ev = explained_variance_score(true, preds)
print(str(n) + '.)' + "\n")
print({'train_precision': train_precision, 
    'train_acc': train_acc, 
    'tr_ev': tr_ev,
    'test_precision': test_precision,
    'test_acc': test_acc, 
    'te_ev': te_ev})

0.25.)

{'train_precision': 1.0, 'train_acc': 0.92537313432835822, 'tr_ev': 0.71703853955375263, 'test_precision': 0.40000000000000002, 'test_acc': 0.50980392156862742, 'te_ev': -0.4104938271604941}


In [50]:
# n = 0.001
len(word_list(0.25))

2822

In [130]:
def get_train_test(all_features, n=0.25):
    all_features, ohe_words = encode_features_and_words(all_features, n)

    shuffle(all_features)

    train_features = all_features[0:n_train]
    test_features = all_features[n_train:n_total]

    X = [f['ohe_feature'] for f in train_features]
    y = [f['ohe_label'] for f in train_features]
    
    x_val = [f['ohe_feature'] for f in test_features]
    y_val = [f['ohe_label'] for f in test_features]
    return [X, y, x_val, y_val]

In [225]:
# 0.25 is a solid value for n. Lower values mean letting in larger amounts of words (features)
# Once validation accuracy is 66.67%, save the weights. That's the max. 
n = 0.25
X, y, x_val, y_val = get_train_test(all_features, n)

In [253]:
shape = len(word_list(n)) + 1

m1 = Sequential([
    BatchNormalization(input_shape=(shape,)),
    Dense(60, activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(30, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(10, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(1, activation='sigmoid')   
])

m1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

sched = [[0.0001, 2], [0.001, 20], [0.01, 10], [0.1, 2], [0.5, 1], [0.1, 5], [0.01, 20], [0.001, 40], [0.0001, 40], [0.00005, 40]]


In [254]:
for info in sched:
    lr, epochs = info
    m1.optimizer.lr = lr
    m1.fit(np.array(X), np.array(y), epochs=epochs,  batch_size=64, validation_data=(np.array(x_val), np.array(y_val)))

Train on 201 samples, validate on 51 samples
Epoch 1/2
201/201 [==============================] - 6s 31ms/step - loss: 0.9273 - acc: 0.4677 - val_loss: 2.2144 - val_acc: 0.5294
Epoch 2/2
201/201 [==============================] - 0s 537us/step - loss: 0.8328 - acc: 0.5224 - val_loss: 1.5095 - val_acc: 0.5294
Train on 201 samples, validate on 51 samples
Epoch 1/20
201/201 [==============================] - 0s 582us/step - loss: 0.8089 - acc: 0.5075 - val_loss: 1.2447 - val_acc: 0.5294
Epoch 2/20
201/201 [==============================] - 0s 542us/step - loss: 0.7648 - acc: 0.5423 - val_loss: 1.0973 - val_acc: 0.5490
Epoch 3/20
201/201 [==============================] - 0s 765us/step - loss: 0.7778 - acc: 0.5124 - val_loss: 1.0068 - val_acc: 0.5490
Epoch 4/20
201/201 [==============================] - 0s 732us/step - loss: 0.7234 - acc: 0.5871 - val_loss: 0.9351 - val_acc: 0.5686
Epoch 5/20
201/201 [==============================] - 0s 631us/step - loss: 0.7399 - acc: 0.5821 - val_loss: 

Epoch 20/20
201/201 [==============================] - 0s 396us/step - loss: 0.6063 - acc: 0.6766 - val_loss: 0.6127 - val_acc: 0.6667
Train on 201 samples, validate on 51 samples
Epoch 1/40
201/201 [==============================] - 0s 398us/step - loss: 0.6265 - acc: 0.6119 - val_loss: 0.6125 - val_acc: 0.6667
Epoch 2/40
201/201 [==============================] - 0s 413us/step - loss: 0.6405 - acc: 0.5871 - val_loss: 0.6127 - val_acc: 0.6667
Epoch 3/40
201/201 [==============================] - 0s 518us/step - loss: 0.6183 - acc: 0.6567 - val_loss: 0.6121 - val_acc: 0.6667
Epoch 4/40
201/201 [==============================] - 0s 666us/step - loss: 0.6038 - acc: 0.6816 - val_loss: 0.6123 - val_acc: 0.6667
Epoch 5/40
201/201 [==============================] - 0s 483us/step - loss: 0.6177 - acc: 0.6517 - val_loss: 0.6142 - val_acc: 0.6667
Epoch 6/40
201/201 [==============================] - 0s 484us/step - loss: 0.6008 - acc: 0.6468 - val_loss: 0.6141 - val_acc: 0.6667
Epoch 7/40
201/2

201/201 [==============================] - 0s 563us/step - loss: 0.5403 - acc: 0.7164 - val_loss: 0.6142 - val_acc: 0.6471
Epoch 21/40
201/201 [==============================] - 0s 609us/step - loss: 0.5407 - acc: 0.6866 - val_loss: 0.6153 - val_acc: 0.6275
Epoch 22/40
201/201 [==============================] - 0s 559us/step - loss: 0.5763 - acc: 0.6816 - val_loss: 0.6152 - val_acc: 0.6275
Epoch 23/40
201/201 [==============================] - 0s 546us/step - loss: 0.4904 - acc: 0.7313 - val_loss: 0.6161 - val_acc: 0.6078
Epoch 24/40
201/201 [==============================] - 0s 596us/step - loss: 0.5199 - acc: 0.7413 - val_loss: 0.6160 - val_acc: 0.6078
Epoch 25/40
201/201 [==============================] - 0s 624us/step - loss: 0.5409 - acc: 0.7313 - val_loss: 0.6171 - val_acc: 0.6078
Epoch 26/40
201/201 [==============================] - 0s 590us/step - loss: 0.5553 - acc: 0.7114 - val_loss: 0.6167 - val_acc: 0.6078
Epoch 27/40
201/201 [==============================] - 0s 544us/ste

201/201 [==============================] - 0s 638us/step - loss: 0.5085 - acc: 0.7413 - val_loss: 0.6009 - val_acc: 0.6667
